<a href="https://colab.research.google.com/github/adt125/AI_project_2023/blob/master/Rule_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk 
import numpy as np
import re
import random
from nltk.stem import wordnet                                  # to perform lemmitization
from sklearn.feature_extraction.text import CountVectorizer    # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer    # to perform tfidf
from nltk import pos_tag                                       # for parts of speech
from sklearn.metrics import pairwise_distances                 # to perfrom cosine similarity
from nltk import word_tokenize                                 # to create tokens
from nltk.corpus import stopwords                              # for stop words
import nltk
import pickle
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
df = pd.read_csv("mentalhealth.csv")
df.head()

,Question_ID,Questions,Answers
0,1590140,What is mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl..."
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...


In [3]:
nltk.download('punkt') 
nltk.download('wordnet')
lemma = wordnet.WordNetLemmatizer() 
# lemma.lemmatize('passed', pos='v') #example

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
nltk.download('averaged_perceptron_tagger') 
nltk.download('stopwords')
stop = stopwords.words('english')
pickle.dump(stop,open('stop.pkl','wb'))
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# function that performs text normalization steps and returns the lemmatized tokens as a sentence

def text_normalization(text):
    text = str(text).lower()                        # text to lower case
    spl_char_text = re.sub(r'[^ a-z]','',text)      # removing special characters
    tokens = nltk.word_tokenize(spl_char_text)      # word tokenizing
    lema = wordnet.WordNetLemmatizer()              # intializing lemmatization
    tags_list = pos_tag(tokens,tagset=None)         # parts of speech
    lema_words = []                                 # empty list 
    for token,pos_token in tags_list:               # lemmatize according to POS
        # print(token)
        if pos_token.startswith('V'):               # Verb
            pos_val = 'v'
        elif pos_token.startswith('J'):             # Adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):             # Adverb
            pos_val = 'r'
        else:
            pos_val = 'n'                           # Noun
        lema_token = lema.lemmatize(token,pos_val)
        
        # if lema_token in stop: 
        lema_words.append(lema_token)             # appending the lemmatized token into a list
    
    return " ".join(lema_words) 

In [6]:
df['lemmatized_text'] = df['Questions'].apply(text_normalization)   # clean text
df.head(5)

,Question_ID,Questions,Answers,lemmatized_text
0,1590140,What is mental illness?,Mental illnesses are health conditions that di...,what be mental illness
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl...",who do mental illness affect
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what be some of the warning sign of mental ill...
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi...",can people with mental illness recover
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...,what should i do if i know someone who appear ...


In [7]:
cv = CountVectorizer()                                  # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [8]:

# returns all the unique word from data 

features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,about,addiction,adhd,adult,advance,affect,after,age,alcohol,allow,...,who,why,with,wont,work,worried,worry,you,young,youth
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
# using tf-idf

tfidf = TfidfVectorizer()                                             # intializing tf-id 
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()        # transforming the data into array

In [10]:
df_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out())
df_tfidf.head()

,about,addiction,adhd,adult,advance,affect,after,age,alcohol,allow,...,who,why,with,wont,work,worried,worry,you,young,youth
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.584795,0.0,0.0,0.0,0.0,...,0.536323,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.411888,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.335099,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
pickle.dump(df_tfidf,open('df_tdidf.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [12]:
def chat_tfidf(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    tf = tfidf.transform([lemma]).toarray() # applying tf-idf
    cos = 1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value = cos.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [13]:
chat_tfidf('what causes mental health issues?')

'Challenges or problems with your mental health can arise from psychological, biological, and social, issues, as well as life events.'

In [14]:
chat_tfidf('How can I feel better?')

'Create a plan for switching to a different treatment that will be a better fit.'

In [15]:

# defining a function that returns response to query using bow

def chat_bow(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    bow = cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value = cosine_value.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [16]:
greet_inp=["hello", "hi", "what's up?","hey" ]
greet_responses=["Hi!","Hello!","Hey there"]
def greet(sentence):
  for word in sentence.split():
    if word.lower() in greet_inp:
      return random.choice(greet_responses)

In [19]:
exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
flag=True
print("Mental Health FAQ")
while(flag==True):
  user_response=input()
  user_response=user_response.lower()
  if(user_response not in exit_commands):
      if(greet(user_response)!=None):
        print(greet(user_response))
        print()
      else:
        print(chat_tfidf(user_response))
        print()
  else:
      flag=False
print("Goodbye! Take Care <3")


Mental Health FAQ
hello
Hello!

where can i get help
family member, friend, clergy, healthcare provider, or other professionals

can we prevent mental problmes
We can all suffer from mental health challenges, but developing our wellbeing, resilience, and seeking help early can help prevent challenges becoming serious.

how to stop overthinking
Create a plan for switching to a different treatment that will be a better fit.

howt to stop worrying
Create a plan for switching to a different treatment that will be a better fit.

how to feel good
Anyone can post anything online. It's up to the readers to think critically about information to decide if it seems helpful, truthful, or safe. Here are key points to keep in mind as you determine which information is worth considering and which information doesn't pass the test. 
 It's so easy to spread information online, and we can't assume that the source we're reading or watching is the original source. Before you can evaluate anything, you nee